### I. Chunk text

In [6]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [1]:
import os
import json

In [11]:
# Đọc file và tách văn bản theo định dạng "số + ."
def read_and_chunk_file(content):
    # Tách văn bản dựa trên các chỉ mục "số + ."
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n", "."], chunk_size=300, chunk_overlap=50
    )
    chunks = text_splitter.split_text(content)
    return chunks

In [12]:
def read_json_files(folder_path):
    visa_chunks = []  # Danh sách để lưu các chunk và metadata

    # Duyệt qua các file trong thư mục
    for filename in os.listdir(folder_path):
        if filename.endswith(".json"):  # Chỉ xử lý các file .json
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                # Lấy thông tin từ mỗi file JSON
                visa_id = data.get("visaID")
                visa_name = data.get("visaName")
                content = data.get("content")

                # Sử dụng hàm read_and_chunk_file để tách nội dung
                chunks = read_and_chunk_file(content)

                # Tạo metadata cho từng chunk
                for chunk in chunks:
                    visa_chunks.append({
                        "text": chunk,
                        "metadata": {
                            "visa_id": visa_id,
                            "visa_name": visa_name
                        }
                    })

    return visa_chunks

In [13]:
# Thư mục chứa các file JSON
folder_path = '../Knowledge_Graph/Data/crawled_v1/'

# Đọc các file JSON và tạo ra metadata và chunks
visa_chunks = read_json_files(folder_path)

# In ra một số kết quả
for chunk in visa_chunks[:5]:  # In ra 5 chunk đầu tiên để kiểm tra
    print(f"Chunk: {chunk['text']}\nMetadata: {chunk['metadata']}\n")

Chunk: 1. Subclass 601
2. Electronic Travel Authority
Up to 3 months at a time
There is a service charge of AUD20 to use the Australian ETA app. There is no other charge payable for an ETA.
Metadata: {'visa_id': 'visa1.1', 'visa_name': 'Electronic Travel Authority (subclass 601)'}

Chunk: In most cases you will be notified of the result of your ETA application immediately. Your application may take longer if you do not complete the application process correctly or we require further information. We recommend that you apply well ahead of when you want to travel.
See all conditions
Metadata: {'visa_id': 'visa1.1', 'visa_name': 'Electronic Travel Authority (subclass 601)'}

Chunk: See all conditions
3. With this visa you can
Travel to Australia:
You can study or train for up to 3 months in total on this visa. If your main reason for travelling to Australia is to study, you should apply for a student visa.
Metadata: {'visa_id': 'visa1.1', 'visa_name': 'Electronic Travel Authority (subclass

In [6]:
# Sử dụng HuggingFaceEmbeddings để mã hóa các chunks
# Thay thế model embedding
embedding = HuggingFaceEmbeddings()
vector_store = Chroma.from_texts(chunks, embedding)

# Đặt câu hỏi và tìm các chunk có độ tương đồng cao nhất
query = "How long i can stay?"
docs_with_scores = vector_store.similarity_search_with_score(query, k=5)

# In ra top 5 chunk có độ tương đồng cao nhất
for doc, score in docs_with_scores:
    print(f"Chunk: {doc}\nScore: {score}\n")

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_62336\3961047193.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings()
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_62336\3961047193.py:3: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedding = HuggingFaceEmbeddings()


: 